In [21]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy

import aot

In [22]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)

settings_root_path = base_dir / core_settings["paths"]["settings_path"]
video_db_path = base_dir / "data/videos/database_originals.tsv"
video_db = pd.read_csv(video_db_path, sep="\t")

main_settings_dir = base_dir / core_settings["paths"]["settings_path"] / "main"
memory_settings_dir = base_dir / core_settings["paths"]["settings_path"] / "memory_main"
subject_number = core_settings["various"]["subject_number"]
run_number = core_settings["various"]["run_number"]
session_number = core_settings["various"]["session_number"]
unique_video_number_per_session = core_settings["various"][
    "unique_video_number_per_session"
]
img_number_per_session = unique_video_number_per_session


# imagenet_folder = '/Users/shufanzhang/Documents/PhD/Arrow_of_time/arrow_of_time/imagenet_sub'
# imagenet_pictures = os.listdir(imagenet_folder)
# unvalide images now because pexels images are not similar to imagenet images


unused_videos = []
"""
for i in range(1, len(video_db)):
    if video_db['grade'][i] == 'k' or video_db['grade'][i] == 'l': 
        unused_videos.append(video_db['video_name'][i])
"""
# should get used videos original name in other way
print("unused videos number: ", len(unused_videos))

imgs_from_unused_videos = []
for i in range(len(unused_videos)):
    imgs_from_unused_videos.append(unused_videos[i].replace(".mp4", ".png"))
imgs_from_unused_videos = [
    "S_" + i for i in imgs_from_unused_videos
] + imagenet_pictures
print("unused images number: ", len(imgs_from_unused_videos))

unused videos number:  385
unused images number:  2623


In [23]:
def gen_memory_design(subject,session):
    print(f"subject: {subject}, session: {session}")
    #read main experiment settings
    main_runs_settings = []
    main_runs_videos = []
    main_runs_videos_name = []#raw video name without .mp4
    for ind in range(run_number):
        run = ind + 1
        runfile = f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}_run_{str(run).zfill(2)}.yml"
        runfile = main_settings_dir / runfile
        runsetting = yaml.load(open(runfile), Loader=yaml.FullLoader)
        main_runs_settings.append(runsetting)
    for runsetting in main_runs_settings:
        raw_video_list = runsetting["stimuli"]["movie_files"]
        video_list = [video for video in raw_video_list if video != "blank"]
        #print(video_list)
        main_runs_videos.append(video_list)
    for videos in main_runs_videos:
        name_list= [video.split(".")[0] for video in videos]
        main_runs_videos_name.append(name_list)
        #print(name_list)
    #flatten the videos list
    flat_video_names = [item for sublist in main_runs_videos_name for item in sublist]
    #remove repeated videos
    flat_video_names = list(set(flat_video_names))
    #print(flat_video_names)
    imgnames = [video + ".png" for video in flat_video_names]
    print(imgnames)
    #save imgnames as settings for memory experiment yaml file
    settings = settings = copy.deepcopy(stimuli_settings_temp)

    half_img_num = int((img_number_per_session)/2)
    #sample half_img_num images from imgnames
    imgnames_used = random.sample(imgnames, half_img_num)
    imgnames_unused = random.sample(imgs_from_unused_videos, half_img_num)
    imgnames_final = imgnames_used + imgnames_unused
    random.shuffle(imgnames_final)

    settings["stimuli"]["picture_files"] = imgnames_final##not right. should subsample from imgnames and add some img from unused videos
    print(settings)
    outfile = f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}_run_01.yml"
    outfile = memory_settings_dir / outfile
    with open(outfile, "w") as f:
        yaml.dump(settings, f)


In [24]:
def gen_memory_desing_for_all():
    for sub in range(subject_number):
        sub = sub + 1
        for ses in range(session_number):
            ses = ses + 1
            gen_memory_design(sub, ses)       

In [25]:
gen_memory_desing_for_all()

subject: 1, session: 1
['S_yt-WhQMF3WM0Ig_29.png', 'R_S_yt-cokedWwwYYs_136.png', 'S_yt--5ls65Ue1RA_52.png', 'S_yt-YeT4acsbqok_210.png', 'R_S_yt-B8Sy53C6Pxw_8.png', 'R_S_yt-GVaF4tmJBZE_253.png', 'R_S_yt-h8wy6x6oytA_25.png', 'S_yt-hZxbKZ1WCVg_24.png', 'R_S_yt-kINZMjys3O4_112.png', 'S_yt-nMWqezngycw_10.png', 'S_yt-pO5SVq-9EhU_114.png', 'S_yt-O_4DgW-XKvA_35.png', 'R_S_yt-6aqg1P9YqR8_38.png', 'R_S_yt-4rwBpRuHg-c_1308.png', 'R_S_yt-_HuBzj8EPBc_62.png', 'R_S__6qAy77ODUI_175.png', 'S_yt-LUIvP_NDWAs_45.png', 'R_S_yt-zam74qlzGtQ_122.png', 'R_S_video-7.png', 'S_yt-EBuE7GbFw7U_180.png', 'R_S_pexels-tima-miroshnichenko-8724308.png', 'S_ILSVRC2015_train_00165011.png', 'R_S_yt-9EnchawJ3pA_90.png', 'S_yt-EuyfT0PuOoQ_91.png', 'S_video-7.png', 'R_S_yt-wEbfIR14zZ8_7.png', 'R_S_yt-EAMpitsjbbA_153.png', 'S_yt-2V2tq9-O_Nc_1669.png', 'R_S_comp_GL_G002_C002_PSNK_v03_SDR_PS_20180925_SDR_2K_AVC.png', 'S_pexels-kindel-media-8165572.png', 'R_S_yt-0xzYSBvVfYM_197.png', 'S_video-2_copy.png', 'R_S_yt-wrfAMDCg93Y_8.p